In [7]:
import openai
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from dotenv import load_dotenv
import os

In [8]:
load_dotenv("/Users/priyal/Documents/honours/.env/key.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

In [9]:
def generate_question(abstract):
    prompt = (
        "Imagine you are a 65-year-old woman looking for simple, practical health advice. "
        "Based on the following abstract, generate a natural-sounding question that an older woman might ask. "
        "Use simple and conversational language. Avoid technical or research-specific phrasing.\n\n"
        f"Abstract: {abstract}\n\n"
        "Example questions:\n"
        "- My knees hurt when I walk. What can I do to make them feel better naturally?\n"
        "- I have trouble sleeping. Are there any natural remedies that could help?\n"
        "- What foods or herbs might help with digestion?\n\n"
        "Generate a similar question based on the abstract above."
    )

    try:
        client = openai.OpenAI()
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=50,
        )
        question = response.choices[0].message.content.strip()
        return question
    except Exception as e:
        print(f"Error generating question: {e}")
        return None

In [10]:
def generate_answer(abstract, title):
    prompt = (
        "Provide a simple, easy-to-understand answer to a 65-year-old woman based on the following abstract. "
        "Use clear and everyday language, avoiding complex medical terms. "
        "Mention the research paper title as a citation in this format: 'According to [Paper Title], ...'.\n\n"
        f"Abstract: {abstract}\n\n"
        f"Paper Title: {title}\n\n"
        "Example answers:\n"
        "- According to 'Herbal Remedies for Joint Pain', turmeric can help with knee pain because it reduces swelling. "
        "You can drink turmeric tea or add a little to warm milk.\n"
        "- 'Natural Sleep Aids' suggests that chamomile tea can help with sleep. Drinking a cup before bed may help you feel more relaxed.\n\n"
        "Now, generate a similar answer based on the abstract above."
    )

    try:
        client = openai.OpenAI()
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7,
            max_tokens=150,
        )
        answer = response.choices[0].message.content.strip()
        return answer
    except Exception as e:
        print(f"Error generating answer: {e}")
        return None

In [13]:
def main(input_csv, output_csv):
    df = pd.read_csv(input_csv)
    df = df.dropna(subset=["Abstract"])
    df = df.head(1000)  

    dataset = []  

    for index, row in df.iterrows():
        title = row["Title"]
        abstract = row["Abstract"]

        question = generate_question(abstract)
        if not question:
            continue

        answer = generate_answer(abstract, title)
        if not answer:
            continue

        example = {"instruction": question, "input": abstract, "output": answer}
        dataset.append(example)

        print(f"✅ Generated Q&A pair {index+1}: {question}")  # New progress print

    print(f"\nProcessed {len(dataset)} QA pairs")
    
    dataset_df = pd.DataFrame(dataset)  
    dataset_df.to_csv(output_csv, index=False)
    print(f"📂 QA dataset saved to {output_csv}")

In [14]:
input_csv = "/Users/priyal/Documents/honours/datasets/extracted_papers_info.csv"
output_csv = "/Users/priyal/Documents/honours/datasets/qa_dataset_bigger_65yo.csv"

main(input_csv, output_csv)

✅ Generated Q&A pair 1: Can Ayurveda practices like changing my diet or reducing stress help improve my health by influencing my genes?
✅ Generated Q&A pair 2: I've been struggling with my diabetes management. Is there a more holistic approach I could try that might help?
✅ Generated Q&A pair 3: I've been having seizures and my current medication isn't helping as much as I'd like. Are there any natural treatments or practices from Ayurveda that might help me manage my epilepsy better?
✅ Generated Q&A pair 4: Can combining traditional Ayurvedic treatments with modern technology help improve my chronic illness?
✅ Generated Q&A pair 5: The abstract provided does not contain any specific information on which to base a health-related question. Please provide a detailed abstract.
✅ Generated Q&A pair 6: I often suffer from migraines. Are there any natural treatments or lifestyle changes from Ayurvedic medicine that could help me prevent them?
✅ Generated Q&A pair 7: Can Ayurvedic techniques 